Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [32]:
batch_size = 128
hidden_size = 1024
graph = tf.Graph()
l2Param = 1e-3
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))
 
  W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  y1 = tf.matmul(tf_train_dataset, W1) + b1
  reluResult = tf.nn.relu(y1)
  
  # Training computation.
  logits = tf.matmul(reluResult, W2) + b2
    
  l2lossW = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2Param * l2lossW)

  print("l2 Loss =", l2loss)
  print("Losst =", loss)

  #regularization
  #loss = tf.nn.l2_loss(loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
  reluR_valid = tf.matmul(y1_valid,W2)+b2
  valid_prediction = tf.nn.softmax(reluR_valid)
    
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
  reluR_test = tf.matmul(y1_test,W2)+b2
  test_prediction = tf.nn.softmax(reluR_test)

num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  print(train_labels.shape)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%% \n" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

l2 Loss = Tensor("add_2:0", shape=(), dtype=float32)
Losst = Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(200000, 10)
Minibatch loss at step 0: 713.761841
Minibatch accuracy: 14.8%
Validation accuracy: 30.5% 

Minibatch loss at step 500: 202.265549
Minibatch accuracy: 76.6%
Validation accuracy: 80.6% 

Minibatch loss at step 1000: 115.418381
Minibatch accuracy: 85.2%
Validation accuracy: 81.8% 

Minibatch loss at step 1500: 68.159088
Minibatch accuracy: 85.9%
Validation accuracy: 83.2% 

Minibatch loss at step 2000: 41.303616
Minibatch accuracy: 86.7%
Validation accuracy: 84.5% 

Minibatch loss at step 2500: 25.350201
Minibatch accuracy: 82.0%
Validation accuracy: 85.7% 

Minibatch loss at step 3000: 15.781467
Minibatch accuracy: 82.8%
Validation accuracy: 86.8% 

Minibatch loss at step 3500: 9.560317
Minibatch accuracy: 86.7%
Validation accuracy: 86.8% 

Minibatch loss at step 4000: 5.863965
Minibatch 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [35]:
batch_size = 128
hidden_size = 1024
graph = tf.Graph()
l2Param = 1e-3

num_bacthes = 3

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))
 
  W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  y1 = tf.matmul(tf_train_dataset, W1) + b1
  reluResult = tf.nn.relu(y1)
  
  # Training computation.
  logits = tf.matmul(reluResult, W2) + b2
    
  l2lossW = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2Param * l2lossW)

  print("l2 Loss =", l2loss)
  print("Losst =", loss)

  #regularization
  #loss = tf.nn.l2_loss(loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
  reluR_valid = tf.matmul(y1_valid,W2)+b2
  valid_prediction = tf.nn.softmax(reluR_valid)
    
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
  reluR_test = tf.matmul(y1_test,W2)+b2
  test_prediction = tf.nn.softmax(reluR_test)

num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  print(train_labels.shape)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step) % num_bacthes
    #print("offset="+str(offset))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%% \n" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

l2 Loss = Tensor("add_2:0", shape=(), dtype=float32)
Losst = Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(200000, 10)
Minibatch loss at step 0: 606.683472
Minibatch accuracy: 9.4%
Validation accuracy: 35.6% 

Minibatch loss at step 500: 191.256226
Minibatch accuracy: 100.0%
Validation accuracy: 66.0% 

Minibatch loss at step 1000: 115.988274
Minibatch accuracy: 100.0%
Validation accuracy: 66.0% 

Minibatch loss at step 1500: 70.341614
Minibatch accuracy: 100.0%
Validation accuracy: 66.1% 

Minibatch loss at step 2000: 42.659012
Minibatch accuracy: 100.0%
Validation accuracy: 66.2% 

Minibatch loss at step 2500: 25.870787
Minibatch accuracy: 100.0%
Validation accuracy: 66.2% 

Minibatch loss at step 3000: 15.689510
Minibatch accuracy: 100.0%
Validation accuracy: 66.5% 

Minibatch loss at step 3500: 9.515215
Minibatch accuracy: 100.0%
Validation accuracy: 67.2% 

Minibatch loss at step 4000: 5.771451
Mini

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
batch_size = 128
hidden_size = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))
 
  W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  y1 = tf.matmul(tf_train_dataset, W1) + b1
  reluResult = tf.nn.relu(y1)
  
  keepRate = tf.placeholder(tf.float32, shape=())
  droppedOutReluResult = tf.nn.dropout(reluResult,keepRate)
  
  # Training computation.
  logits = tf.matmul(reluResult, W2) + b2
    
  l2loss = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  print("l2 Loss =", l2loss)
  print("Losst =", loss)

  #regularization
  #loss = tf.nn.l2_loss(loss)
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss )
  optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

    #  0.08 * l2loss
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
  reluR_valid = tf.matmul(y1_valid,W2)+b2
  valid_prediction = tf.nn.softmax(reluR_valid)
    
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
  reluR_test = tf.matmul(y1_test,W2)+b2
  test_prediction = tf.nn.softmax(reluR_test)

num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  print(train_labels.shape)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%% \n" % accuracy(
        valid_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), test_labels))

l2 Loss = Tensor("add_2:0", shape=(), dtype=float32)
Losst = Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(200000, 10)
Minibatch loss at step 0: 303.206665
Minibatch accuracy: 8.6%
Validation accuracy: 8.6% 

Minibatch loss at step 500: 41.102390
Minibatch accuracy: 67.2%
Validation accuracy: 64.9% 

Minibatch loss at step 1000: 43.370583
Minibatch accuracy: 78.1%
Validation accuracy: 71.3% 

Minibatch loss at step 1500: 35.700085
Minibatch accuracy: 77.3%
Validation accuracy: 73.9% 

Minibatch loss at step 2000: 34.781322
Minibatch accuracy: 72.7%
Validation accuracy: 75.2% 

Minibatch loss at step 2500: 41.747314
Minibatch accuracy: 71.1%
Validation accuracy: 76.3% 

Minibatch loss at step 3000: 40.237747
Minibatch accuracy: 74.2%
Validation accuracy: 77.1% 

Minibatch loss at step 3500: 26.195248
Minibatch accuracy: 80.5%
Validation accuracy: 77.2% 

Minibatch loss at step 4000: 22.314754
Minibatch ac


 ---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 ---


In [24]:
batch_size = 128
hidden_size = 1024
graph = tf.Graph()
l2Param = 1e-3
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))
 
  W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  y1 = tf.matmul(tf_train_dataset, W1) + b1
  reluResult = tf.nn.relu(y1)
  
  keepRate = tf.placeholder(tf.float32, shape=())
  droppedOutReluResult = tf.nn.dropout(reluResult,keepRate)
  
  # Training computation.
  logits = tf.matmul(reluResult, W2) + b2
    
  l2lossW = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2Param * tf.nn.l2_loss(l2lossW))

  print("l2 Loss =", l2loss)
  print("Losst =", loss)

  #regularization
  #loss = tf.nn.l2_loss(loss)
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss )
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 200, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
    
  #optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

    #  0.08 * l2loss
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
  reluR_valid = tf.matmul(y1_valid,W2)+b2
  valid_prediction = tf.nn.softmax(reluR_valid)
    
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
  reluR_test = tf.matmul(y1_test,W2)+b2
  test_prediction = tf.nn.softmax(reluR_test)

num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  print(train_labels.shape)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%% \n" % accuracy(
        valid_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keepRate:1.0}), test_labels))

l2 Loss = Tensor("add_2:0", shape=(), dtype=float32)
Losst = Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(200000, 10)
Minibatch loss at step 0: 49519776.000000
Minibatch accuracy: 10.9%
Validation accuracy: 12.6% 

Minibatch loss at step 500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0% 

Minibatch loss at step 1000: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0% 

Minibatch loss at step 1500: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0% 

Minibatch loss at step 2000: nan
Minibatch accuracy: 6.2%
Validation accuracy: 10.0% 

Minibatch loss at step 2500: nan
Minibatch accuracy: 10.2%
Validation accuracy: 10.0% 

Minibatch loss at step 3000: nan
Minibatch accuracy: 7.8%
Validation accuracy: 10.0% 

Minibatch loss at step 3500: nan
Minibatch accuracy: 6.2%
Validation accuracy: 10.0% 

Minibatch loss at step 4000: nan
Minibatch accuracy: 10.9%
Validation accuracy: 10.0% 

Min

 ---
Using muilt-layers.
 ---

In [10]:
batch_size = 128
hidden_size1 = 1024
hidden_size2 = 100
graph = tf.Graph()
with graph.as_default():
    
  global_step = tf.Variable(0)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size1]))
  b1 = tf.Variable(tf.zeros([hidden_size1]))
    
  W2 = tf.Variable(tf.truncated_normal([hidden_size1, hidden_size2]))
  b2 = tf.Variable(tf.zeros([hidden_size2]))
 
  W3 = tf.Variable(tf.truncated_normal([hidden_size2, num_labels]))
  b3 = tf.Variable(tf.zeros([num_labels]))
  
    
  y1_train = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
  y2_train = tf.nn.relu(tf.matmul(y1_train, W2) + b2)
  logits = tf.matmul(y2_train, W3) + b3
        
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+
      1e-3 * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3)))

  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
  y2_valid = tf.nn.relu(tf.matmul(y1_valid, W2) + b2)
  valid_prediction = tf.nn.softmax(tf.matmul(y2_valid,W3)+b3)
    
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
  y2_test = tf.nn.relu(tf.matmul(y1_test, W2) + b2)
  test_prediction = tf.nn.softmax(tf.matmul(y2_test,W3)+b3)

num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  print(train_labels.shape)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%% \n" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(200000, 10)
Minibatch loss at step 0: 2282.113770
Minibatch accuracy: 4.7%
Validation accuracy: 11.3% 

Minibatch loss at step 500: 6954979055108096.000000
Minibatch accuracy: 9.4%
Validation accuracy: 10.0% 

Minibatch loss at step 1000: 4217883102019584.000000
Minibatch accuracy: 10.9%
Validation accuracy: 10.0% 

Minibatch loss at step 1500: 2557954746220544.000000
Minibatch accuracy: 11.7%
Validation accuracy: 10.0% 

Minibatch loss at step 2000: 1551284205256704.000000
Minibatch accuracy: 6.2%
Validation accuracy: 10.0% 

Minibatch loss at step 2500: 940783863595008.000000
Minibatch accuracy: 10.2%
Validation accuracy: 10.0% 

Minibatch loss at step 3000: 570542985838592.000000
Minibatch accuracy: 10.2%
Validation accuracy: 10.0% 

Minibatch loss at step 3500: 346008672272384.000000
Minibatch accuracy: 7.0%
Validation accuracy: 10.0% 

Minibatch loss at step 4000: 209838680309760.000000
Minibatc

# stddev 加在W上似乎很重要呢

In [9]:
batch_size = 128
hidden_size1 = 1024
hidden_size2 = 100
graph = tf.Graph()
with graph.as_default():
    
  global_step = tf.Variable(0)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
  W1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, hidden_size1],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    )
  b1 = tf.Variable(tf.zeros([hidden_size1]))
    
  W2 = tf.Variable(
    tf.truncated_normal([hidden_size1, hidden_size2], stddev=np.sqrt(2.0 / hidden_size1)))
  b2 = tf.Variable(tf.zeros([hidden_size2]))
 
  W3 = tf.Variable(
        tf.truncated_normal(
            [hidden_size2, num_labels], 
            stddev=np.sqrt(2.0 / hidden_size2)))
  b3 = tf.Variable(tf.zeros([num_labels]))
  
    
  y1_train = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
  y2_train = tf.nn.relu(tf.matmul(y1_train, W2) + b2)
  logits = tf.matmul(y2_train, W3) + b3
        
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+
      1e-3 * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3)))

  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
      
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
  y2_valid = tf.nn.relu(tf.matmul(y1_valid, W2) + b2)
  valid_prediction = tf.nn.softmax(tf.matmul(y2_valid,W3)+b3)
    
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
  y2_test = tf.nn.relu(tf.matmul(y1_test, W2) + b2)
  test_prediction = tf.nn.softmax(tf.matmul(y2_test,W3)+b3)

num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  print(train_labels.shape)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%% \n" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(200000, 10)
Minibatch loss at step 0: 3.265903
Minibatch accuracy: 9.4%
Validation accuracy: 42.7% 

Minibatch loss at step 500: 1.029091
Minibatch accuracy: 83.6%
Validation accuracy: 85.7% 

Minibatch loss at step 1000: 0.722621
Minibatch accuracy: 89.1%
Validation accuracy: 85.9% 

Minibatch loss at step 1500: 0.595629
Minibatch accuracy: 89.1%
Validation accuracy: 87.2% 

Minibatch loss at step 2000: 0.591512
Minibatch accuracy: 89.1%
Validation accuracy: 87.9% 

Minibatch loss at step 2500: 0.745219
Minibatch accuracy: 84.4%
Validation accuracy: 87.5% 

Minibatch loss at step 3000: 0.731848
Minibatch accuracy: 82.0%
Validation accuracy: 87.6% 

Minibatch loss at step 3500: 0.572114
Minibatch accuracy: 85.9%
Validation accuracy: 86.4% 

Minibatch loss at step 4000: 0.398932
Minibatch accuracy: 92.2%
Validation accuracy: 88.2% 

Minibatch loss at step 4500: 0.440000
Minibatch accuracy: 89.1%
Valid